<a href="https://colab.research.google.com/github/JimmyFerCla/MySql-employeesUT/blob/main/Crisp_dm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-cloud-language

In [ ]:
### **Librerias Proyecto**

In [4]:
#lib textos
from transformers import RobertaTokenizerFast, EncoderDecoderModel
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from string import punctuation

#lib clasicas
import warnings
import torch
import tensorflow as tf
import pandas as pd
import torch
import nltk
import unicodedata
import numpy as np
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

#lib google NLP
import os
from google.cloud import language_v1

from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

In [5]:
import pandas as pd

In [13]:
df = pd.read_excel('/content/HELP_DESK_2.xlsx')

In [14]:
df

,Identicación de la Petición,Proceso,Tipo - Causal de Requerimiento,Fecha,Prioridad,Asignado a,Estado
0,RQ01289,OPERACIONES,"Asignación, inactivación, modificación de Usua...",2023-11-14,Alto,Olber Jimenez,En progreso
1,RQ01288,OPERACIONES,Novedad en impresora,2023-11-14,Medio,Henry Tibocha,Cerrado
2,RQ01287,ADMINISTRACION Y FINANZAS,Novedad en Aplicativo TMS,2023-11-14,Alto,Olber Jimenez,En progreso
3,RQ01286,OPERACIONES,Novedad en impresora,2023-11-14,Medio,Henry Tibocha,En progreso
4,RQ01285,OPERACIONES,Novedad en Aplicativo TMS,NaT,Alto,Olber Jimenez,Cerrado
...,...,...,...,...,...,...,...
1262,RQ00728,NaN,NaN,NaT,NaN,NaN,En progreso
1263,RQ00729,NaN,NaN,NaT,NaN,NaN,En progreso
1264,RQ00730,NaN,NaN,NaT,NaN,NaN,En progreso
1265,RQ00731,NaN,NaN,NaT,NaN,NaN,En progreso


In [5]:
nltk.download('stopwords')
stemmer = SnowballStemmer('spanish')
nltk.download('punkt')

stop_words = set(stopwords.words('spanish'))
stop_words = stop_words.union(set(['cin', 'me', 'le', 'da', 'mi', 'su', 'ha', 'he', 'ya', 'un', 'una', 'es','del', 'las', 'los', 'en', 'que', 'y', 'la','de']))

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

def fast_preproc(text):
    text = text.lower()
    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c not in punctuation)
    text = remove_accents(text)
    words = word_tokenize(text)
    words = [stemmer.stem(word) for word in words]
    words = [word for word in words if not word in stop_words]
    try:
        text = " ".join(str(word) for word in words)
    except Exception as e:
        print(e)
        pass
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
df = pd.read_excel('/content/HELP_DESK_2.xlsx')
df.columns=["Identicacion","Proceso","Tipo","Fecha","Prioridad","Asignado","Estado"]
print("tamaño de los datos:", df.shape)
df.head(5)

tamaño de los datos: (1267, 7)


,Identicacion,Proceso,Tipo,Fecha,Prioridad,Asignado,Estado
0,RQ01289,OPERACIONES,"Asignación, inactivación, modificación de Usua...",2023-11-14,Alto,Olber Jimenez,En progreso
1,RQ01288,OPERACIONES,Novedad en impresora,2023-11-14,Medio,Henry Tibocha,Cerrado
2,RQ01287,ADMINISTRACION Y FINANZAS,Novedad en Aplicativo TMS,2023-11-14,Alto,Olber Jimenez,En progreso
3,RQ01286,OPERACIONES,Novedad en impresora,2023-11-14,Medio,Henry Tibocha,En progreso
4,RQ01285,OPERACIONES,Novedad en Aplicativo TMS,NaT,Alto,Olber Jimenez,Cerrado


In [10]:
df.isna().sum()
df.isnull().sum()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1267 entries, 0 to 1266
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Identicacion  1267 non-null   object        
 1   Proceso       1247 non-null   object        
 2   Tipo          1247 non-null   object        
 3   Fecha         605 non-null    datetime64[ns]
 4   Prioridad     1247 non-null   object        
 5   Asignado      1245 non-null   object        
 6   Estado        1267 non-null   object        
dtypes: datetime64[ns](1), object(6)
memory usage: 69.4+ KB


,Identicacion,Proceso,Tipo,Fecha,Prioridad,Asignado,Estado
count,1267,1247,1247,605,1247,1245,1267
unique,1267,11,24,66,3,4,5
top,RQ01289,OPERACIONES,Novedad en Aplicativo TMS,2023-10-17 00:00:00,Alto,Olber Jimenez,Cerrado
freq,1,646,217,27,667,596,1070
first,NaN,NaN,NaN,2023-09-01 00:00:00,NaN,NaN,NaN
last,NaN,NaN,NaN,2023-11-14 00:00:00,NaN,NaN,NaN
